### Install env

In [ ]:
!pip install gensim

In [ ]:
!wget https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextskipgram_300_5_2018.tgz

In [7]:
!tar -xzf araneum_none_fasttextskipgram_300_5_2018.tgz

In [ ]:
!pip install sklearn-crfsuite

In [14]:
!git clone https://github.com/king-menin/mipt-nlp2021.git

In [ ]:
!pip install tensorboardX

Data from https://github.com/dialogue-evaluation/factRuEval-2016

In [15]:
!ls mipt-nlp2021/seminars/sem4/

sem4_ner.ipynb	src  test.csv  train.csv


### Run exps
FactRuEval-2016 competition of NER
http://www.dialog-21.ru/evaluation/2016/letter/



In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("mipt-nlp2021/seminars/sem4/")

In [3]:
import gensim

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


In [4]:
model = gensim.models.KeyedVectors.load('araneum_none_fasttextskipgram_300_5_2018.model')

INFO:gensim.utils:loading Word2VecKeyedVectors object from araneum_none_fasttextskipgram_300_5_2018.model
INFO:gensim.utils:loading vectors_ngrams from araneum_none_fasttextskipgram_300_5_2018.model.vectors_ngrams.npy with mmap=None
INFO:gensim.utils:loading vectors from araneum_none_fasttextskipgram_300_5_2018.model.vectors.npy with mmap=None
INFO:gensim.utils:loading vectors_vocab from araneum_none_fasttextskipgram_300_5_2018.model.vectors_vocab.npy with mmap=None
INFO:gensim.utils:setting ignored attribute vectors_vocab_norm to None
INFO:gensim.utils:setting ignored attribute vectors_ngrams_norm to None
INFO:gensim.utils:setting ignored attribute vectors_norm to None
INFO:gensim.utils:setting ignored attribute buckets_word to None
INFO:gensim.utils:loaded araneum_none_fasttextskipgram_300_5_2018.model


In [2]:
from src.learner import NerLearner, validate_step

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("mipt-nlp2021/seminars/sem4/train.csv", sep="\t")

In [7]:
df.head()

,labels,text,clf
0,O O O O O O B_ORG I_ORG O B_LOC I_LOC O O O O ...,В понедельник 28 июня у здания мэрии Москвы на...,False
1,O O O O O O O O O O O B_LOC B_PER I_PER O O O ...,"Среди требований , выдвигаемых организаторами ...",False
2,O O O O O O O O O O O O O O O O O O O O B_PER ...,Участникам акции предлагалось принести с собой...,False
3,O O O O O O O O O O O O O O O O O,Начало акции было намечено на 19 часов ; подчё...,True
4,O O O O O O O O O O O O O B_LOC I_LOC I_LOC O ...,"Освещающие акцию блоггеры сообщили , что автоб...",False


In [8]:
learn = NerLearner(
    train_df_path="mipt-nlp2021/seminars/sem4/train.csv",
    valid_df_path="mipt-nlp2021/seminars/sem4/test.csv",
    embedder=model,
    tensorboard_dir="logs",
    hidden_dim=1024,
    lr=0.0004,
    use_embeds=False
)

In [9]:
sum([p.numel() for p in learn.model.parameters() if p.requires_grad])

3863148

In [10]:
learn.model

BiLSTMCRF(
  (lstm): BiLSTM(
    (dropout): Dropout(p=0.3, inplace=False)
    (lstm): LSTM(300, 512, batch_first=True, bidirectional=True)
  )
  (crf): CRFDecoder(
    (input_dropout): Dropout(p=0.5, inplace=False)
    (linear): Linears(
      (linears): ModuleList(
        (0): Linear(in_features=1024, out_features=512, bias=True)
      )
      (output_linear): Linear(in_features=512, out_features=8, bias=True)
    )
    (crf): CRF()
  )
)

In [ ]:
# for b in learn.data.train_dl:
#     break

In [ ]:
# learn.model.forward(b)

In [ ]:
learn.fit(10)

In [ ]:
print(validate_step(learn.data.valid_dl, learn.model, learn.data.train_ds.idx2label))

              precision    recall  f1-score   support

       B_LOC      0.333     0.014     0.026      1324
       B_ORG      0.513     0.118     0.191      1905
       B_PER      0.245     0.017     0.032      1335
       I_LOC      0.286     0.009     0.017       233
       I_ORG      0.478     0.137     0.213      1960
       I_PER      0.291     0.055     0.093       777
           O      0.880     0.993     0.933     48875

    accuracy                          0.870     56409
   macro avg      0.432     0.192     0.215     56409
weighted avg      0.816     0.870     0.825     56409



### Try to learn embeddings

In [ ]:
learn = NerLearner(
    train_df_path="mipt-nlp2021/seminars/sem4/train.csv",
    valid_df_path="mipt-nlp2021/seminars/sem4/test.csv",
    embedder=model,
    tensorboard_dir="logs",
    hidden_dim=1024,
    lr=0.0004,
    use_embeds=True
)

In [ ]:
sum([p.numel() for p in learn.model.parameters() if p.requires_grad])

10562448

In [ ]:
learn.model

BiLSTMCRF(
  (embeddings): Embedding(22331, 300)
  (lstm): BiLSTM(
    (dropout): Dropout(p=0.3, inplace=False)
    (lstm): LSTM(300, 512, batch_first=True, bidirectional=True)
  )
  (crf): CRFDecoder(
    (input_dropout): Dropout(p=0.5, inplace=False)
    (linear): Linears(
      (linears): ModuleList(
        (0): Linear(in_features=1024, out_features=512, bias=True)
      )
      (output_linear): Linear(in_features=512, out_features=8, bias=True)
    )
    (crf): CRF()
  )
)

In [ ]:
learn.fit(10)

In [ ]:
print(validate_step(learn.data.valid_dl, learn.model, learn.data.train_ds.idx2label))

              precision    recall  f1-score   support

       <pad>      0.000     0.000     0.000         0
       B_LOC      0.678     0.461     0.549      1324
       B_ORG      0.544     0.169     0.257      1905
       B_PER      0.597     0.172     0.267      1335
       I_LOC      0.232     0.056     0.090       233
       I_ORG      0.404     0.135     0.203      1960
       I_PER      0.591     0.230     0.331       777
           O      0.900     0.986     0.941     48875

    accuracy                          0.883     56409
   macro avg      0.493     0.276     0.330     56409
weighted avg      0.852     0.883     0.855     56409

